In [ ]:
# import libraries

from collections import deque
from time import time
import datetime
import json
from glob import glob

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.rnn.python.ops.rnn_cell import PhasedLSTMCell
from tensorflow.python.ops.rnn import dynamic_rnn

In [ ]:
# methods - mostly from training.py

def compute_returns(close_prices):
    close_prices_returns = pd.DataFrame(100 * ((close_prices.shift(-1) - close_prices) / close_prices).fillna(0.0))
    return close_prices_returns.shift(1).fillna(0)


def get_batch(bs, prices, sequence_length):
    batch_x = []
    batch_t = []
    batch_y = []
    for jj in range(bs):
        start = np.random.choice(range(len(prices) - sequence_length - 1))
        values = prices[start: start + sequence_length + 1].values
        x = np.array(values[0:-1, 1], dtype=float)
        y = np.array(values[-1, 1], dtype=float)
        t = np.array(values[0:-1, 0], dtype=float)
        batch_x.append(x)
        batch_t.append(t)
        batch_y.append(y)
    return np.expand_dims(batch_x, axis=2), np.expand_dims(batch_t, axis=2), np.expand_dims(batch_y, axis=1)


def run_training(hidden_size, batch_size, steps, num_layers=1, return_only_last_output=True):

    ####################### MODEL PART #######################
    sequence_length = 20  # for now let's do like this.
    learning_rate = 1e-7
    print('hidden_size:', hidden_size)
    print('num_layers:', num_layers)
    print('batch_size:', batch_size)
    print('steps:', steps)
    print('learning_rate:', learning_rate)
    print('sequence_length:', sequence_length)

    x_ = tf.placeholder(tf.float32, (batch_size, sequence_length, 1))
    t_ = tf.placeholder(tf.float32, (batch_size, sequence_length, 1))
    y_ = tf.placeholder(tf.float32, (batch_size, 1))

    
    # multi_lstm - combines training.py & models > model_helper.py
    rnn_out = []
    for i in range(num_layers):
        x, _ = dynamic_rnn(cell=PhasedLSTMCell(hidden_size), inputs=(t_, x_), dtype=tf.float32, scope='LSTM_' + str(np.random.random_sample()))
    if return_only_last_output == True:
        rnn_out = tf.squeeze(x[:, -1, :])
    else:
        rnn_out = x        

    out = slim.fully_connected(inputs=rnn_out,
                               num_outputs=hidden_size,
                               activation_fn=tf.nn.tanh)

    out = slim.fully_connected(inputs=out,
                               num_outputs=1,
                               activation_fn=None)

    print('*' * 80)
    print('TRAINABLE VARIABLES')
    for tv in tf.trainable_variables():
        print(tv)
    num_params = np.sum([np.prod([int(e) for e in d.shape.dims], axis=0) for d in tf.trainable_variables()])
    print('TOTAL NUMBER OF TRAINABLE VARIABLES = {}'.format(num_params))
    print('*' * 80)

    loss = 100 * tf.reduce_mean(tf.abs(out - y_))
    benchmark_loss = 100 * tf.reduce_mean(tf.abs(y_))
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)  # clip please.

    sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
    sess.run(tf.global_variables_initializer())

    ####################### DATA PART #######################
    # removing the columns where the last price did not move. It biases the model.
    prices = read_price_data_example()
    prices = prices[['timestamp', 'last']].astype(np.float)
    prices['last'] = compute_returns(prices['last'])
    prices = prices[prices['last'] != 0]

    ####################### RUN PART #######################
    running_difference = deque(maxlen=100)
    running_accuracy = deque(maxlen=100)
    for i in range(steps):
        x_train, t_train, y_train = get_batch(batch_size, prices, sequence_length)
        st = time()
        sess.run([train_step], feed_dict={x_: x_train, y_: y_train, t_: t_train})  # gradient update.

        x_test, t_test, y_test = get_batch(batch_size, prices, sequence_length)
        te_loss, be_loss = sess.run([loss, benchmark_loss],
                                    feed_dict={x_: x_test, y_: y_test, t_: t_test})
        running_difference.append(be_loss - te_loss)
        running_accuracy.append(te_loss < be_loss)
        print(
            'steps = {0} | time {1:.3f} | te_loss = {2:.6f}, be_loss = {3:.6f}, r_diff = {4:.6f}, r_acc = {5:.3f}'.format(
                str(i).zfill(6), time() - st, te_loss, be_loss, np.mean(running_difference), np.mean(running_accuracy)))
        
# not used yet - from data > read_price_data.py
def read_price_data():
    
    HEADERS = ['high', 'last', 'timestamp', 'bid', 'vwap', 'volume', 'low', 'ask', 'open']

    np.set_printoptions(threshold=np.nan)
    pd.set_option('display.height', 1000) # height has been deprecated
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)
        
    np_data = []
    all_json = glob('bitstamp_record_price' + '/*.json')
    print('Found {} prices updates.'.format(len(all_json)))
    # bar = progressbar.ProgressBar()
    for filename in all_json:
        # print(filename)
        try:
            with open(filename, 'r') as r:
                d = json.load(r)
                l = []
                for header in HEADERS:
                    l.append(str(d[header]))
            np_data.append(l)
        except:
            print('Problem with filename [{}].'.format(filename))

    if len(np_data) == 0:
        raise Exception('No data available in {}'.format(data_dir))

    np_data = np.array(np_data)
    d = pd.DataFrame(np_data, index=np_data[:, 2])
    d.columns = HEADERS
    d.index = d.index.map(lambda ts: datetime.datetime.fromtimestamp(int(ts)))
    print('Data set has {} rows.'.format(len(d)))
    d.drop_duplicates(inplace=True)
    print('Removing duplicates...')
    print('Data set has {} rows.'.format(len(d)))
    d.index.names = ['DateTime_UTC']
    d.to_csv(arg_p.output_file)
    print(d)
    return d

# data actually used
def read_price_data_example():
    
    d = pd.read_csv('../data_examples/btc_price_2017-09-13T03:45:28+00:00.csv')
    return d

In [ ]:
# run

run_training(hidden_size=1024, batch_size=32, steps=10000, num_layers=3)